# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
!pip install gmaps

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_df = pd.read_csv("../WeatherPy/output_data/cities2.csv")
city_weather_df


,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Lüderitz,-26.6481,15.1594,66.42,77,0,11.07,NaN,1613253100
1,Cape Town,-33.9258,18.4232,70.00,94,20,4.61,ZA,1613253060
2,Vilhena,-12.7406,-60.1458,75.20,88,75,9.22,BR,1613253100
3,Leningradskiy,69.3833,178.4167,-12.30,86,100,1.88,RU,1613253100
4,Okhotsk,59.3833,143.3000,13.21,96,100,2.42,RU,1613253101
...,...,...,...,...,...,...,...,...,...
567,Söderhamn,61.3037,17.0592,28.99,68,100,3.00,SE,1613253241
568,Onda,39.9667,-0.2500,55.00,67,47,4.29,ES,1613253242
569,Santa Cruz de la Sierra,-17.8000,-63.1667,87.80,54,40,11.50,BO,1613253242
570,Kostanay,53.1667,63.5833,26.60,80,0,13.42,KZ,1613253242


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = city_weather_df[["Lat", "Lon"]].astype(float)
humid = city_weather_df["Humidity"].astype(float)

max_humid = city_weather_df["Humidity"].max()
max_humid

100

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
clean_weather_df = city_weather_df.loc[(city_weather_df["Max Temp"] <= 85) & (city_weather_df["Max Temp"] >= 70)\
                                       & (city_weather_df["Cloudiness"] <= 30) & (city_weather_df["Humidity"] <50)\
                                       & (city_weather_df)]
clean_weather_df

TypeError: Cannot perform 'rand_' with a dtyped [float64] array and scalar of type [bool]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:

hotel_df = clean_weather_df[["City", "Country", "Lat", "Lon"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df


,City,Country,Lat,Lon,Hotel Name
37,Hong Kong,HK,22.2855,114.1577,
71,Busselton,AU,-33.6500,115.3333,
107,Ouadda,CF,8.0777,22.4007,
130,Tezpur,IN,26.6333,92.8000,
191,Malakal,SS,9.5334,31.6605,
201,Kibala,TD,9.1104,18.3463,
232,Kalas,IN,18.1667,74.8000,
243,San Rafael,AR,-34.6177,-68.3301,
257,Bouna,CI,9.2667,-3.0000,
261,Chatrapur,IN,19.3500,84.9833,


In [32]:
# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius" : 5000,
    "type": "lodging",
    "key": g_key
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    lat = row['Lat']
    lon = row['Lon']

    # add keyword to params dict
    params['location'] = f"{lat},{lon}"

    # assemble url and make API request
   
    response = requests.get(base_url, params=params).json()
    

    
    try:
        
        
        hotel_df.loc[index, 'Hotel Name'] = response[0]['name']
       
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
